In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
PATH = "data/dogbreed/"

In [4]:
sz = 256
arch = inceptionresnet_2
bs = 32
# Jeremy tried 224, 58 and 299, 58

In [5]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

In [6]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.pivot_table(index='breed', aggfunc=len).sort_values('id', ascending=False)

In [7]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                        val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

## Precompute

In [8]:
data = get_data(sz, bs)

A Jupyter Widget

In [9]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 324/324 [06:10<00:00,  1.14s/it]


In [10]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.5184   0.34153  0.90416]                        
[ 1.       0.39128  0.33756  0.90269]                        



In [11]:
learn.fit(1e-2, 3)

A Jupyter Widget

[ 0.       0.34406  0.33289  0.9036 ]                        
[ 1.      0.3073  0.348   0.9036]                            
[ 2.       0.25114  0.33486  0.91295]                        



## Augment

In [12]:
from sklearn import metrics

In [13]:
data = get_data(sz, bs)

A Jupyter Widget

In [14]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [15]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.57741  0.3468   0.9013 ]                        
[ 1.       0.42838  0.32876  0.90367]                        



In [16]:
learn.precompute=False
learn.bn_freeze=True

In [17]:
learn.fit(1e-2, 5, cycle_len=1)

A Jupyter Widget

[ 0.       0.43426  0.30978  0.90953]                        
[ 1.       0.40703  0.31561  0.91099]                        
[ 2.       0.39883  0.31612  0.91295]                        
[ 3.       0.37582  0.31406  0.90513]                        
[ 4.       0.39464  0.31199  0.90806]                        



In [18]:
learn.save('224_ir2_pre')

In [19]:
learn.load('224_ir2_pre')

## Increase size

In [20]:
learn.set_data(get_data(350, bs))
learn.freeze()

In [21]:
learn.fit(1e-2, 3, cycle_len=1)

A Jupyter Widget

[ 0.       0.26425  0.20561  0.94085]                        
[ 1.       0.23699  0.20112  0.9375 ]                        
[ 2.       0.25931  0.20431  0.93555]                        



In [22]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.2512   0.20465  0.94092]                        
[ 1.       0.23679  0.21176  0.93897]                        
[ 2.       0.21673  0.2053   0.93701]                        
[ 3.       0.24298  0.22057  0.93555]                        
[ 4.       0.2336   0.20631  0.94141]                        
[ 5.       0.21032  0.20672  0.93945]                        
[ 6.       0.23118  0.20698  0.93945]                        



In [23]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds, y), metrics.log_loss(y, probs)

(0.94227005870841485, 0.19324874883224669)

In [25]:
learn.save('350_ir2_pre')

In [26]:
learn.load('350_ir2_pre')

In [27]:
learn.fit(1e-2, 1, cycle_len=2)

A Jupyter Widget

[ 0.       0.22714  0.21479  0.93701]                        
[ 1.       0.18911  0.20874  0.93603]                        



In [28]:
learn.save('350_ir2_pre')

In [29]:
log_preds, y = learn.TTA(is_test=True)
probs_ir2 = np.exp(log_preds)
#accuracy(log_preds, y), metrics.log_loss(y, probs)

In [30]:
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()

In [31]:
save_array('probs_ir2.bc', probs_ir2)

In [32]:
learn.save('350_ir2_pre')

## Use whole dataset now

In [31]:
learn.load('299_pre')

In [32]:
def get_data_whole(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                        val_idxs=[0], suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

In [33]:
def load_array(fname): return bcolz.open(fname)[:]

In [34]:
data = get_data_whole(sz, bs)

A Jupyter Widget

In [35]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.freeze()

100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


In [36]:
learn.fit(1e-2, 5)

A Jupyter Widget

[ 0.       0.75786  0.01884  1.     ]                         
[ 1.       0.38308  0.00202  1.     ]                         
[ 2.       0.27529  0.00073  1.     ]                         
[ 3.       0.22052  0.00052  1.     ]                         
[ 4.       0.19112  0.00022  1.     ]                         



In [37]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [38]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.94042  0.00524  1.     ]                         
[ 1.       0.48962  0.00265  1.     ]                         



In [39]:
learn.precompute = False
learn.bn_freeze = True

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.44712  0.00127  1.     ]                        
[ 1.       0.39743  0.00121  1.     ]                        
[ 2.       0.34848  0.00075  1.     ]                        
[ 3.       0.35622  0.00137  1.     ]                        
[ 4.       0.31755  0.00057  1.     ]                        
[ 5.       0.28701  0.00074  1.     ]                        
 76%|███████▌  | 134/177 [03:41<01:10,  1.65s/it, loss=0.29] 

In [ ]:
learn.set_data(get_data_whole(299, bs))
learn.freeze()

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA(is_test=True)
probs_resnx101_64_full = np.exp(log_preds)
#accuracy(log_preds, y), metrics.log_loss(y, probs)

In [ ]:
save_array('probs_resnx101_64_full.bc', probs_resnx101_64_full)

In [ ]:
learn.save('299_full')

## Try ensembling
### Use another arch first

## Save to file for submission

In [33]:
df = pd.DataFrame(probs_ir2)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [34]:
df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,fd1a7be32f10493735555e62913c0841,2.101389e-06,7.957380e-07,9.901231e-07,7.696650e-07,4.725467e-07,3.137218e-07,1.815584e-07,1.709327e-07,4.579634e-07,...,2.395443e-06,2.345402e-06,8.872038e-07,2.088811e-06,6.659964e-08,3.753928e-07,3.749388e-07,1.273274e-06,4.798866e-07,3.512119e-08
1,5273d42f02b4715cb93845205995ef64,1.615717e-05,1.668898e-05,2.292900e-06,3.470073e-06,1.326146e-06,9.195463e-07,6.441616e-08,8.183958e-07,1.369859e-06,...,2.356505e-06,5.158620e-07,1.096716e-06,6.168519e-06,5.028619e-06,3.254074e-06,1.100742e-06,2.905195e-06,8.317512e-06,5.890093e-07
2,442057a3142f4d75a1023db363e2fb54,7.130004e-07,8.126507e-07,2.995882e-07,1.080347e-06,3.360327e-07,8.103186e-06,6.558137e-08,9.681415e-08,7.517328e-06,...,2.840640e-07,7.324162e-07,4.261292e-06,5.974223e-05,9.524963e-07,1.255658e-07,1.527046e-06,1.828355e-07,7.973781e-07,5.305102e-07
3,fb03302b030afd122e0f4936c158f6c1,6.470754e-05,5.273749e-06,7.179740e-07,6.052573e-06,2.462417e-05,4.126265e-06,2.563426e-05,1.258198e-05,2.768924e-05,...,7.947544e-05,1.446267e-05,7.264930e-06,1.506572e-05,1.073374e-05,2.775627e-06,9.247089e-06,2.588709e-06,6.563337e-06,2.558486e-04
4,511ca6e3539192aa5c8747bb697e0525,8.379841e-07,1.200375e-06,5.936035e-07,7.524296e-07,2.809074e-06,4.269319e-07,1.336144e-05,2.859274e-07,1.571820e-06,...,4.072642e-06,9.853200e-07,2.454565e-07,2.628426e-07,1.317447e-06,4.162125e-07,2.284507e-05,4.138440e-07,1.315784e-05,9.399819e-07


In [35]:
SUBM = f'{PATH}results_2/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}submission11.csv', index=False)
#df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]

In [ ]:
fn

In [ ]:
Image.open(PATH+fn).resize((150, 150))

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

In [ ]:
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
im = trn_tfms(Image.open(PATH+fn))
preds = to_np(learn.model(V(T(im[None]).cude())))
np.argmax(preds)

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

In [ ]:
im = trn_tfms(Image.open(PATH+fn))
preds = learn.predict_array(im[None])
np.argmax(preds)

## Confusion matrix

In [ ]:
preds = np.argmax(log_preds, axis=1)
probs = np.exp(log_preds[:,1])

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)